In [1]:
import json
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from vgc_project.r import create_R_model_interface, ImmutableDataFrame
from vgc_project.parameter_fit import create_fit_vgc_model_to_trials, Trial

joblib_cache_location = "./_analysiscache"
rmods = create_R_model_interface(joblib_cache_location=joblib_cache_location)
fit_vgc_model_to_trials = create_fit_vgc_model_to_trials(joblib_cache_location=joblib_cache_location)

import analysisutils
from analysisutils import predictor_names, short_predictor_names
from prep_data import \
    mazes,\
    model_preds,\
    get_exp5a_at, get_exp2_5a_at, \
    get_exp5a_at, get_exp2_5a_at, \
    get_exp3_5b_at__exp3_5b_mt,\
    get_exp5b_at__exp5b_mt,\
    get_exp6a_navdist,\
    get_exp1_navdist,\
    get_exp6a_at,\
    get_exp1_6a_at,\
    get_exp6b_at__exp6b_mt,\
    get_exp3_at__exp3_mt,\
    get_exp3_6b_at__exp3_6b_mt,\
    get_exp1_at,\
    get_exp2_at,\
    get_exp3_at__exp3_mt,\
    get_exp5a_at,\
    get_exp5b_at__exp5b_mt,\
    get_exp6a_at,\
    get_exp6b_at__exp6b_mt

R[write to console]: Loading required package: Matrix



In [2]:
def llr_test_string_summary(row):
    return f"$\chi^2({int(row['Df']):d}) = " + \
        f"{row['Chisq']:.2f}" + \
        ", p " + analysisutils.pval_to_string(row["Pr(>Chisq)"]) + "$"

In [3]:
%load_ext rpy2.ipython

In [4]:
%%R
library(lme4)
library(lmerTest)
library(lmtest)
library(scales)

Attaching package: ‘lmerTest’



    lmer



    step



Attaching package: ‘zoo’



    as.Date, as.Date.numeric




# Experiment 5a/5b - Perceptual controls

In [5]:
exp5a_at = get_exp5a_at()
exp2_5a_at = get_exp2_5a_at()

## Experiment 5a

In [6]:
%%R -i exp2_5a_at -o exp2_5a_interaction_test,exp2_5a_vgc_maineff_sum,exp2_5a_vgc_int_sum
# Test if there is an interaction between experiment and vgc

# use sum coding for experiment comparison
exp2_5a_at$exp <- factor(exp2_5a_at$exp)
contrasts(exp2_5a_at$exp) <- contr.sum(2)
print(contrasts(exp2_5a_at$exp))
exp2_5a_interaction <- lmer(
    attention_N ~ (1 | sessionId) + (1 | grid) +
        exp*vgc_weight_Z
    ,
    data=exp2_5a_at,
    control=lmerControl(optimizer="bobyqa",optCtrl=list(maxfun=1e+07)),
    REML=F
)
exp2_5a_interaction_lesioned <- update(exp2_5a_interaction, ~ . - exp:vgc_weight_Z)

# test if removing vgc x exp interactoin is sig
exp2_5a_interaction_test <- lrtest(
    exp2_5a_interaction_lesioned,
    exp2_5a_interaction
)

# get resulting coefficients from model
exp2_5a_interaction_summary <- summary(exp2_5a_interaction)
exp2_5a_vgc_maineff_sum <- paste(
    "$\\beta = ", round(exp2_5a_interaction_summary$coefficients["vgc_weight_Z",]["Estimate"], digits=3),
    ", \\text{S.E.} = ", round(exp2_5a_interaction_summary$coefficients["vgc_weight_Z",]["Std. Error"], digits=3),
    "$", sep=""
)
exp2_5a_vgc_int_sum <- paste(
    "$\\beta = ", round(exp2_5a_interaction_summary$coefficients["exp1:vgc_weight_Z",]["Estimate"], digits=3),
    ", \\text{S.E.} = ", round(exp2_5a_interaction_summary$coefficients["exp1:vgc_weight_Z",]["Std. Error"], digits=3),
    "$", sep=""
)

      [,1]
exp2     1
exp5a   -1


In [7]:
with open("./inputs/exp2_5a_interaction_test.tex", "w") as f:
    exp2_5a_interaction_test_str = llr_test_string_summary(exp2_5a_interaction_test.iloc[1])
    f.write(exp2_5a_interaction_test_str)
    print("exp2_5a_interaction: ", exp2_5a_interaction_test_str)
    
with open("./inputs/exp2_5a_interaction_vgc_maineff.tex", "w") as f:
    f.write(exp2_5a_vgc_maineff_sum[0])
    print("exp2_5a_vgc_maineff_sum: ", exp2_5a_vgc_maineff_sum[0])
    
with open("./inputs/exp2_5a_interaction_vgc_interaction.tex", "w") as f:
    f.write(exp2_5a_vgc_int_sum[0])
    print("exp2_5a_vgc_int_sum: ", exp2_5a_vgc_int_sum[0])

exp2_5a_interaction:  $\chi^2(1) = 950.19, p = 1.2 \times 10^{-208}$
exp2_5a_vgc_maineff_sum:  $\beta = 0.057, \text{S.E.} = 0.002$
exp2_5a_vgc_int_sum:  $\beta = 0.052, \text{S.E.} = 0.002$


## Experiment 5b

In [8]:
exp3_5b_at, exp3_5b_mt = get_exp3_5b_at__exp3_5b_mt()
exp5b_at, exp5b_mt = get_exp5b_at__exp5b_mt()

In [9]:
%%R -i exp3_5b_at,exp3_5b_mt
# use sum coding for experiment comparison
exp3_5b_at$exp <- factor(exp3_5b_at$exp)
contrasts(exp3_5b_at$exp) <- contr.sum(2)
print(contrasts(exp3_5b_at$exp))

exp3_5b_mt$exp <- factor(exp3_5b_mt$exp)
contrasts(exp3_5b_mt$exp) <- contr.sum(2)
print(contrasts(exp3_5b_mt$exp))



      [,1]
exp3     1
exp5b   -1
      [,1]
exp3     1
exp5b   -1


In [10]:
%%R -o exp3_5b_acc_interaction_test,exp3_5b_acc_vgc_maineff_sum,exp3_5b_acc_vgc_int_sum
# accuracy
exp3_5b_acc_interaction <- glmer(
    correct ~ (1 | sessionId) + (1 | grid) +
        exp*vgc_weight_Z
    ,
    data=exp3_5b_mt,
    control=glmerControl(optimizer="bobyqa",optCtrl=list(maxfun=1e+07)),
    family=binomial
)
exp3_5b_acc_interaction_lesioned <- update(exp3_5b_acc_interaction, ~ . - exp:vgc_weight_Z)

# test if removing vgc x exp interactoin is sig
exp3_5b_acc_interaction_test <- lrtest(
    exp3_5b_acc_interaction_lesioned,
    exp3_5b_acc_interaction
)

# get resulting coefficients from model
exp3_5b_acc_interaction_summary <- summary(exp3_5b_acc_interaction)
exp3_5b_acc_vgc_maineff_sum <- paste(
    "$\\beta = ", round(exp3_5b_acc_interaction_summary$coefficients["vgc_weight_Z",]["Estimate"], digits=3),
    ", \\text{S.E.} = ", round(exp3_5b_acc_interaction_summary$coefficients["vgc_weight_Z",]["Std. Error"], digits=3),
    "$", sep=""
)
exp3_5b_acc_vgc_int_sum <- paste(
    "$\\beta = ", round(exp3_5b_acc_interaction_summary$coefficients["exp1:vgc_weight_Z",]["Estimate"], digits=3),
    ", \\text{S.E.} = ", round(exp3_5b_acc_interaction_summary$coefficients["exp1:vgc_weight_Z",]["Std. Error"], digits=3),
    "$", sep=""
)


In [11]:
with open("./inputs/exp3_5b_acc_interaction_test.tex", "w") as f:
    exp3_5b_acc_interaction_test_str = llr_test_string_summary(exp3_5b_acc_interaction_test.iloc[1])
    f.write(exp3_5b_acc_interaction_test_str)
    print("exp3_5b_acc_interaction_test: ", exp3_5b_acc_interaction_test_str)
    
with open("./inputs/exp3_5b_acc_vgc_maineff.tex", "w") as f:
    f.write(exp3_5b_acc_vgc_maineff_sum[0])
    print("exp3_5b_acc_vgc_maineff_sum: ", exp3_5b_acc_vgc_maineff_sum[0])
    
with open("./inputs/exp3_5b_acc_vgc_interaction.tex", "w") as f:
    f.write(exp3_5b_acc_vgc_int_sum[0])
    print("exp3_5b_acc_vgc_int_sum: ", exp3_5b_acc_vgc_int_sum[0])

exp3_5b_acc_interaction_test:  $\chi^2(1) = 51.50, p = 7.2 \times 10^{-13}$
exp3_5b_acc_vgc_maineff_sum:  $\beta = 0.443, \text{S.E.} = 0.027$
exp3_5b_acc_vgc_int_sum:  $\beta = 0.195, \text{S.E.} = 0.027$


In [12]:
%%R -o exp3_5b_conf_interaction_test,exp3_5b_conf_vgc_maineff_sum,exp3_5b_conf_vgc_int_sum
# confidence
exp3_5b_conf_interaction <- lmer(
    conf_N ~ (1 | sessionId) + (1 | grid) +
        exp*vgc_weight_Z
    ,
    data=exp3_5b_mt,
    control=lmerControl(optimizer="bobyqa",optCtrl=list(maxfun=1e+07)),
    REML=F
)
exp3_5b_conf_interaction_lesioned <- update(exp3_5b_conf_interaction, ~ . - exp:vgc_weight_Z)
# test if removing vgc x exp interactoin is sig
exp3_5b_conf_interaction_test <- lrtest(
    exp3_5b_conf_interaction_lesioned,
    exp3_5b_conf_interaction
)

# get resulting coefficients from model
exp3_5b_conf_interaction_summary <- summary(exp3_5b_conf_interaction)
exp3_5b_conf_vgc_maineff_sum <- paste(
    "$\\beta = ", round(exp3_5b_conf_interaction_summary$coefficients["vgc_weight_Z",]["Estimate"], digits=3),
    ", \\text{S.E.} = ", round(exp3_5b_conf_interaction_summary$coefficients["vgc_weight_Z",]["Std. Error"], digits=3),
    "$", sep=""
)
exp3_5b_conf_vgc_int_sum <- paste(
    "$\\beta = ", round(exp3_5b_conf_interaction_summary$coefficients["exp1:vgc_weight_Z",]["Estimate"], digits=3),
    ", \\text{S.E.} = ", round(exp3_5b_conf_interaction_summary$coefficients["exp1:vgc_weight_Z",]["Std. Error"], digits=3),
    "$", sep=""
)

In [13]:
with open("./inputs/exp3_5b_conf_interaction_test.tex", "w") as f:
    exp3_5b_conf_interaction_test_str = llr_test_string_summary(exp3_5b_conf_interaction_test.iloc[1])
    f.write(exp3_5b_conf_interaction_test_str)
    print("exp3_5b_conf_interaction_test: ", exp3_5b_conf_interaction_test_str)
    
with open("./inputs/exp3_5b_conf_vgc_maineff.tex", "w") as f:
    f.write(exp3_5b_conf_vgc_maineff_sum[0])
    print("exp3_5b_conf_vgc_maineff_sum: ", exp3_5b_conf_vgc_maineff_sum[0])
    
with open("./inputs/exp3_5b_conf_vgc_interaction.tex", "w") as f:
    f.write(exp3_5b_conf_vgc_int_sum[0])
    print("exp3_5b_conf_vgc_int_sum: ", exp3_5b_conf_vgc_int_sum[0])

exp3_5b_conf_interaction_test:  $\chi^2(1) = 176.35, p = 3.0 \times 10^{-40}$
exp3_5b_conf_vgc_maineff_sum:  $\beta = 0.064, \text{S.E.} = 0.003$
exp3_5b_conf_vgc_int_sum:  $\beta = 0.04, \text{S.E.} = 0.003$


In [14]:
%%R -o exp3_5b_attn_interaction_test,exp3_5b_attn_vgc_maineff_sum,exp3_5b_attn_vgc_int_sum

# attention
exp3_5b_attn_interaction <- lmer(
    attention_N ~ (1 | sessionId) + #(1 | grid) + #is singular with by-maze random effect
        exp*vgc_weight_Z
    ,
    data=exp3_5b_at,
    control=lmerControl(optimizer="bobyqa",optCtrl=list(maxfun=1e+07)),
    REML=F
)
exp3_5b_attn_interaction_lesioned <- update(exp3_5b_attn_interaction, ~ . - exp:vgc_weight_Z)
# test if removing vgc x exp interactoin is sig
exp3_5b_attn_interaction_test <- lrtest(
    exp3_5b_attn_interaction_lesioned,
    exp3_5b_attn_interaction
)

# get resulting coefficients from model
exp3_5b_attn_interaction_summary <- summary(exp3_5b_attn_interaction)
exp3_5b_attn_vgc_maineff_sum <- paste(
    "$\\beta = ", round(exp3_5b_attn_interaction_summary$coefficients["vgc_weight_Z",]["Estimate"], digits=3),
    ", \\text{S.E.} = ", round(exp3_5b_attn_interaction_summary$coefficients["vgc_weight_Z",]["Std. Error"], digits=3),
    "$", sep=""
)
exp3_5b_attn_vgc_int_sum <- paste(
    "$\\beta = ", round(exp3_5b_attn_interaction_summary$coefficients["exp1:vgc_weight_Z",]["Estimate"], digits=3),
    ", \\text{S.E.} = ", round(exp3_5b_attn_interaction_summary$coefficients["exp1:vgc_weight_Z",]["Std. Error"], digits=3),
    "$", sep=""
)

In [15]:
with open("./inputs/exp3_5b_attn_interaction_test.tex", "w") as f:
    exp3_5b_attn_interaction_test_str = llr_test_string_summary(exp3_5b_attn_interaction_test.iloc[1])
    f.write(exp3_5b_attn_interaction_test_str)
    print("exp3_5b_attn_interaction_test: ", exp3_5b_attn_interaction_test_str)
    
with open("./inputs/exp3_5b_attn_vgc_maineff.tex", "w") as f:
    f.write(exp3_5b_attn_vgc_maineff_sum[0])
    print("exp3_5b_attn_vgc_maineff_sum: ", exp3_5b_attn_vgc_maineff_sum[0])
    
with open("./inputs/exp3_5b_attn_vgc_interaction.tex", "w") as f:
    f.write(exp3_5b_attn_vgc_int_sum[0])
    print("exp3_5b_attn_vgc_int_sum: ", exp3_5b_attn_vgc_int_sum[0])

exp3_5b_attn_interaction_test:  $\chi^2(1) = 675.93, p = 5.1 \times 10^{-149}$
exp3_5b_attn_vgc_maineff_sum:  $\beta = 0.09, \text{S.E.} = 0.003$
exp3_5b_attn_vgc_int_sum:  $\beta = 0.086, \text{S.E.} = 0.003$


# Experiment 6a/6b - Execution controls

## Experiment 6a

In [16]:
exp6a_navdist = get_exp6a_navdist()
exp1_navdist = get_exp1_navdist()
exp6a_at = get_exp6a_at()
exp1_6a_at = get_exp1_6a_at()

Loading Experiment 1 Attention Trials


In [17]:
#check that the mean navdists are almost the same
from scipy import stats 
nav_mindists = pd.concat([
    exp6a_navdist.groupby(['grid', 'obstacle'])['nav_mindist'].mean(),
    exp1_navdist.groupby(['grid', 'obstacle'])['nav_mindist'].mean()
], axis=1)
nav_mindists.columns = ['exp6a', 'exp1']
corr, pval = stats.pearsonr(nav_mindists.exp6a, nav_mindists.exp1)
exp1_6a_navdist_corr_sum = \
    f"$r({len(nav_mindists)}) = {corr:.2f}, p {analysisutils.pval_to_string(pval)}$"
# sns.scatterplot(data=nav_mindists, x='exp6a', y='exp1')
assert .99 < corr < 1.0
with open("./inputs/exp1_6a_navdist_corr.tex", "w") as file:
    file.write(exp1_6a_navdist_corr_sum)
print(exp1_6a_navdist_corr_sum)

r(84) = 1.00, p = 1.5 \times 10^{-139}


In [18]:
%%R -i exp1_6a_at -o exp1_6a_interaction_test,exp1_6a_vgc_maineff_sum,exp1_6a_vgc_int_sum
# use sum coding for experiment comparison
exp1_6a_at$exp <- factor(exp1_6a_at$exp)
contrasts(exp1_6a_at$exp) <- contr.sum(2)
print(contrasts(exp1_6a_at$exp))

exp1_6a_interaction <- lmer(
    attention_N ~ (1 | sessionId) + (1 | grid) + 
        exp*vgc_weight_Z 
    ,
    data=exp1_6a_at,
    control=lmerControl(optimizer="bobyqa",optCtrl=list(maxfun=1e+07)),
    REML=F
)
exp1_6a_interaction_lesioned <- update(exp1_6a_interaction, ~ . - exp:vgc_weight_Z)
# test if removing vgc x exp interactoin is sig
exp1_6a_interaction_test <- lrtest(
    exp1_6a_interaction_lesioned,
    exp1_6a_interaction
)

# get resulting coefficients from model
exp1_6a_interaction_summary <- summary(exp1_6a_interaction)
exp1_6a_vgc_maineff_sum <- paste(
    "$\\beta = ", round(exp1_6a_interaction_summary$coefficients["vgc_weight_Z",]["Estimate"], digits=3),
    ", \\text{S.E.} = ", round(exp1_6a_interaction_summary$coefficients["vgc_weight_Z",]["Std. Error"], digits=3),
    "$", sep=""
)
exp1_6a_vgc_int_sum <- paste(
    "$\\beta = ", round(exp1_6a_interaction_summary$coefficients["exp1:vgc_weight_Z",]["Estimate"], digits=3),
    ", \\text{S.E.} = ", round(exp1_6a_interaction_summary$coefficients["exp1:vgc_weight_Z",]["Std. Error"], digits=3),
    "$", sep=""
)

      [,1]
exp1     1
exp6a   -1


In [19]:
with open("./inputs/exp1_6a_interaction_test.tex", "w") as f:
    exp1_6a_interaction_test_str = llr_test_string_summary(exp1_6a_interaction_test.iloc[1])
    f.write(exp1_6a_interaction_test_str)
    print("exp1_6a_interaction: ", exp1_6a_interaction_test_str)
    
with open("./inputs/exp1_6a_interaction_vgc_maineff.tex", "w") as f:
    f.write(exp1_6a_vgc_maineff_sum[0])
    print("exp1_6a_vgc_maineff_sum: ", exp1_6a_vgc_maineff_sum[0])
    
with open("./inputs/exp1_6a_interaction_vgc_interaction.tex", "w") as f:
    f.write(exp1_6a_vgc_int_sum[0])
    print("exp1_6a_vgc_int_sum: ", exp1_6a_vgc_int_sum[0])

exp1_6a_interaction:  $\chi^2(1) = 120.89, p = 4.0 \times 10^{-28}$
exp1_6a_vgc_maineff_sum:  $\beta = 0.114, \text{S.E.} = 0.002$
exp1_6a_vgc_int_sum:  $\beta = 0.019, \text{S.E.} = 0.002$


## Experiment 6b

In [20]:
exp6b_at, exp6b_mt = get_exp6b_at__exp6b_mt()
exp3_at, exp3_mt = get_exp3_at__exp3_mt()
exp3_6b_at, exp3_6b_mt = get_exp3_6b_at__exp3_6b_mt()

In [21]:
#check that the mean navdists are almost the same
from scipy import stats 
nav_mindists = pd.concat([
    pd.concat([
        exp6b_at[['grid', 'obstacle', 'nav_mindist']],
        exp6b_mt[['grid', 'obstacle', 'nav_mindist']],
    ], axis=0).groupby(['grid', 'obstacle'])['nav_mindist'].mean(),
    pd.concat([
        exp3_at[['grid', 'obstacle', 'nav_mindist']],
        exp3_mt[['grid', 'obstacle', 'nav_mindist']],
    ], axis=0).groupby(['grid', 'obstacle'])['nav_mindist'].mean(),
], axis=1)
nav_mindists.columns = ['exp6b', 'exp3']
corr, pval = stats.pearsonr(nav_mindists.exp6b, nav_mindists.exp3)
exp3_6b_navdist_corr_sum = \
    f"$r({len(nav_mindists)}) = {corr:.2f}, p {analysisutils.pval_to_string(pval)}$"
# sns.scatterplot(data=nav_mindists, x='exp6b', y='exp3')
assert .99 < corr < 1.0
with open("./inputs/exp3_6b_navdist_corr.tex", "w") as file:
    file.write(exp3_6b_navdist_corr_sum)
print(exp3_6b_navdist_corr_sum)

r(20) = 1.00, p = 2.9 \times 10^{-24}


In [22]:
%%R -i exp3_6b_at,exp3_6b_mt
# use sum coding for experiment comparison
exp3_6b_at$exp <- factor(exp3_6b_at$exp)
contrasts(exp3_6b_at$exp) <- contr.sum(2)
print(contrasts(exp3_6b_at$exp))

exp3_6b_mt$exp <- factor(exp3_6b_mt$exp)
contrasts(exp3_6b_mt$exp) <- contr.sum(2)
print(contrasts(exp3_6b_mt$exp))

      [,1]
exp3     1
exp6b   -1
      [,1]
exp3     1
exp6b   -1


In [23]:
%%R -o exp3_6b_acc_interaction_test,exp3_6b_acc_vgc_maineff_sum,exp3_6b_acc_vgc_int_sum
# accuracy
exp3_6b_acc_interaction <- glmer(
    correct ~ (1 | sessionId) + (1 | grid) +
        exp*vgc_weight_Z
    ,
    data=exp3_6b_mt,
    control=glmerControl(optimizer="bobyqa",optCtrl=list(maxfun=1e+07)),
    family=binomial
)
exp3_6b_acc_interaction_lesioned <- update(exp3_6b_acc_interaction, ~ . - exp:vgc_weight_Z)

# test if removing vgc x exp interactoin is sig
exp3_6b_acc_interaction_test <- lrtest(
    exp3_6b_acc_interaction_lesioned,
    exp3_6b_acc_interaction
)

# get resulting coefficients from model
exp3_6b_acc_interaction_summary <- summary(exp3_6b_acc_interaction)
exp3_6b_acc_vgc_maineff_sum <- paste(
    "$\\beta = ", round(exp3_6b_acc_interaction_summary$coefficients["vgc_weight_Z",]["Estimate"], digits=3),
    ", \\text{S.E.} = ", round(exp3_6b_acc_interaction_summary$coefficients["vgc_weight_Z",]["Std. Error"], digits=3),
    "$", sep=""
)
exp3_6b_acc_vgc_int_sum <- paste(
    "$\\beta = ", round(exp3_6b_acc_interaction_summary$coefficients["exp1:vgc_weight_Z",]["Estimate"], digits=3),
    ", \\text{S.E.} = ", round(exp3_6b_acc_interaction_summary$coefficients["exp1:vgc_weight_Z",]["Std. Error"], digits=3),
    "$", sep=""
)


In [24]:
with open("./inputs/exp3_6b_acc_interaction_test.tex", "w") as f:
    exp3_6b_acc_interaction_test_str = llr_test_string_summary(exp3_6b_acc_interaction_test.iloc[1])
    f.write(exp3_6b_acc_interaction_test_str)
    print("exp3_6b_acc_interaction_test: ", exp3_6b_acc_interaction_test_str)
    
with open("./inputs/exp3_6b_acc_vgc_maineff.tex", "w") as f:
    f.write(exp3_6b_acc_vgc_maineff_sum[0])
    print("exp3_6b_acc_vgc_maineff_sum: ", exp3_6b_acc_vgc_maineff_sum[0])
    
with open("./inputs/exp3_6b_acc_vgc_interaction.tex", "w") as f:
    f.write(exp3_6b_acc_vgc_int_sum[0])
    print("exp3_6b_acc_vgc_int_sum: ", exp3_6b_acc_vgc_int_sum[0])

exp3_6b_acc_interaction_test:  $\chi^2(1) = 60.41, p = 7.7 \times 10^{-15}$
exp3_6b_acc_vgc_maineff_sum:  $\beta = 0.42, \text{S.E.} = 0.028$
exp3_6b_acc_vgc_int_sum:  $\beta = 0.215, \text{S.E.} = 0.028$


In [25]:
%%R -o exp3_6b_conf_interaction_test,exp3_6b_conf_vgc_maineff_sum,exp3_6b_conf_vgc_int_sum
# confidence
exp3_6b_conf_interaction <- lmer(
    conf_N ~ (1 | sessionId) + (1 | grid) +
        exp*vgc_weight_Z
    ,
    data=exp3_6b_mt,
    control=lmerControl(optimizer="bobyqa",optCtrl=list(maxfun=1e+07)),
    REML=F
)
exp3_6b_conf_interaction_lesioned <- update(exp3_6b_conf_interaction, ~ . - exp:vgc_weight_Z)
# test if removing vgc x exp interactoin is sig
exp3_6b_conf_interaction_test <- lrtest(
    exp3_6b_conf_interaction_lesioned,
    exp3_6b_conf_interaction
)

# get resulting coefficients from model
exp3_6b_conf_interaction_summary <- summary(exp3_6b_conf_interaction)
exp3_6b_conf_vgc_maineff_sum <- paste(
    "$\\beta = ", round(exp3_6b_conf_interaction_summary$coefficients["vgc_weight_Z",]["Estimate"], digits=3),
    ", \\text{S.E.} = ", round(exp3_6b_conf_interaction_summary$coefficients["vgc_weight_Z",]["Std. Error"], digits=3),
    "$", sep=""
)
exp3_6b_conf_vgc_int_sum <- paste(
    "$\\beta = ", round(exp3_6b_conf_interaction_summary$coefficients["exp1:vgc_weight_Z",]["Estimate"], digits=3),
    ", \\text{S.E.} = ", round(exp3_6b_conf_interaction_summary$coefficients["exp1:vgc_weight_Z",]["Std. Error"], digits=3),
    "$", sep=""
)

In [26]:
with open("./inputs/exp3_6b_conf_interaction_test.tex", "w") as f:
    exp3_6b_conf_interaction_test_str = llr_test_string_summary(exp3_6b_conf_interaction_test.iloc[1])
    f.write(exp3_6b_conf_interaction_test_str)
    print("exp3_6b_conf_interaction_test: ", exp3_6b_conf_interaction_test_str)
    
with open("./inputs/exp3_6b_conf_vgc_maineff.tex", "w") as f:
    f.write(exp3_6b_conf_vgc_maineff_sum[0])
    print("exp3_6b_conf_vgc_maineff_sum: ", exp3_6b_conf_vgc_maineff_sum[0])
    
with open("./inputs/exp3_6b_conf_vgc_interaction.tex", "w") as f:
    f.write(exp3_6b_conf_vgc_int_sum[0])
    print("exp3_6b_conf_vgc_int_sum: ", exp3_6b_conf_vgc_int_sum[0])

exp3_6b_conf_interaction_test:  $\chi^2(1) = 59.39, p = 1.3 \times 10^{-14}$
exp3_6b_conf_vgc_maineff_sum:  $\beta = 0.078, \text{S.E.} = 0.003$
exp3_6b_conf_vgc_int_sum:  $\beta = 0.025, \text{S.E.} = 0.003$


In [27]:
%%R -o exp3_6b_attn_interaction_test,exp3_6b_attn_vgc_maineff_sum,exp3_6b_attn_vgc_int_sum

# attention
exp3_6b_attn_interaction <- lmer(
    attention_N ~ (1 | sessionId) + (1 | grid) + 
        exp*vgc_weight_Z
    ,
    data=exp3_6b_at,
    control=lmerControl(optimizer="bobyqa",optCtrl=list(maxfun=1e+07)),
    REML=F
)
exp3_6b_attn_interaction_lesioned <- update(exp3_6b_attn_interaction, ~ . - exp:vgc_weight_Z)
# test if removing vgc x exp interactoin is sig
exp3_6b_attn_interaction_test <- lrtest(
    exp3_6b_attn_interaction_lesioned,
    exp3_6b_attn_interaction
)

# get resulting coefficients from model
exp3_6b_attn_interaction_summary <- summary(exp3_6b_attn_interaction)
exp3_6b_attn_vgc_maineff_sum <- paste(
    "$\\beta = ", round(exp3_6b_attn_interaction_summary$coefficients["vgc_weight_Z",]["Estimate"], digits=3),
    ", \\text{S.E.} = ", round(exp3_6b_attn_interaction_summary$coefficients["vgc_weight_Z",]["Std. Error"], digits=3),
    "$", sep=""
)
exp3_6b_attn_vgc_int_sum <- paste(
    "$\\beta = ", round(exp3_6b_attn_interaction_summary$coefficients["exp1:vgc_weight_Z",]["Estimate"], digits=3),
    ", \\text{S.E.} = ", round(exp3_6b_attn_interaction_summary$coefficients["exp1:vgc_weight_Z",]["Std. Error"], digits=3),
    "$", sep=""
)

In [28]:
with open("./inputs/exp3_6b_attn_interaction_test.tex", "w") as f:
    exp3_6b_attn_interaction_test_str = llr_test_string_summary(exp3_6b_attn_interaction_test.iloc[1])
    f.write(exp3_6b_attn_interaction_test_str)
    print("exp3_6b_attn_interaction_test: ", exp3_6b_attn_interaction_test_str)
    
with open("./inputs/exp3_6b_attn_vgc_maineff.tex", "w") as f:
    f.write(exp3_6b_attn_vgc_maineff_sum[0])
    print("exp3_6b_attn_vgc_maineff_sum: ", exp3_6b_attn_vgc_maineff_sum[0])
    
with open("./inputs/exp3_6b_attn_vgc_interaction.tex", "w") as f:
    f.write(exp3_6b_attn_vgc_int_sum[0])
    print("exp3_6b_attn_vgc_int_sum: ", exp3_6b_attn_vgc_int_sum[0])

exp3_6b_attn_interaction_test:  $\chi^2(1) = 97.10, p = 6.6 \times 10^{-23}$
exp3_6b_attn_vgc_maineff_sum:  $\beta = 0.139, \text{S.E.} = 0.004$
exp3_6b_attn_vgc_int_sum:  $\beta = 0.037, \text{S.E.} = 0.004$


# Experiment 5a/5b/6a/6b - Regressing onto both controls

In [29]:
exp1_at = get_exp1_at()
exp2_at = get_exp2_at()
exp3_at, exp3_mt = get_exp3_at__exp3_mt()
exp5a_at = get_exp5a_at()
exp5b_at, exp5b_mt = get_exp5b_at__exp5b_mt()
exp6a_at = get_exp6a_at()
exp6b_at, exp6b_mt = get_exp6b_at__exp6b_mt()

In [30]:
grid_0_11_at_controls = exp5a_at.groupby(['grid', 'obstacle'])['attention_N'].mean().reset_index(name='perceptual_attn').merge(
        exp6a_at.groupby(['grid', 'obstacle'])['attention_N'].mean().reset_index(name='navigation_attn'),
        on=['grid', 'obstacle']
    )
exp1_control_at = exp1_at.merge(
    grid_0_11_at_controls,
    on=['grid', 'obstacle']
)
exp2_control_at = exp2_at.merge(
    grid_0_11_at_controls,
    on=['grid', 'obstacle']
)
grid_12_15_at_controls = exp5b_at.groupby(['grid', 'obstacle'])['attention_N'].mean().reset_index(name='perceptual_attn').merge(
    exp6b_at.groupby(['grid', 'obstacle'])['attention_N'].mean().reset_index(name='navigation_attn'),
    on=['grid', 'obstacle']
)
grid_12_15_mt_controls = exp5b_mt.groupby(['grid', 'obstacle'])[['conf_N', 'correct']].mean().rename(
    columns={'conf_N': 'perceptual_conf', 'correct': 'perceptual_correct'}
).merge(
    exp6b_mt.groupby(['grid', 'obstacle'])[['conf_N', 'correct']].mean().rename(
        columns={'conf_N': 'navigation_conf', 'correct': 'navigation_correct'}
    ),
    on=['grid', 'obstacle']
)
exp3_control_at = exp3_at.\
    merge(grid_12_15_at_controls, on=['grid', 'obstacle']).\
    merge(grid_12_15_mt_controls, on=['grid', 'obstacle'])
exp3_control_mt = exp3_mt.\
    merge(grid_12_15_mt_controls, on=['grid', 'obstacle']).\
    merge(grid_12_15_at_controls, on=['grid', 'obstacle'])

### Exp 1

In [31]:
%%R -i exp1_control_at -o exp1_control_vgc_test,exp1_control_vgc_maineff_sum
exp1_control_vgc <- lmer(
    attention_N ~ (1 | sessionId) + (1 | grid)
        + perceptual_attn + navigation_attn 
        + vgc_weight_Z,
    data=exp1_control_at,
    control=lmerControl(optimizer="bobyqa",optCtrl=list(maxfun=1e+07)),
    REML=F
)
exp1_control_vgc_lesioned <- update(exp1_control_vgc, ~ . - vgc_weight_Z)
# test if lesion is significant
exp1_control_vgc_test <- lrtest(
    exp1_control_vgc_lesioned,
    exp1_control_vgc
)

# get beta on vgc weight
exp1_control_vgc_summary <- summary(exp1_control_vgc)
exp1_control_vgc_maineff_sum <- paste(
    "$\\beta = ", round(exp1_control_vgc_summary$coefficients["vgc_weight_Z",]["Estimate"], digits=3),
    ", \\text{S.E.} = ", round(exp1_control_vgc_summary$coefficients["vgc_weight_Z",]["Std. Error"], digits=3),
    "$", sep=""
)

In [32]:
with open("./inputs/exp1_control_vgc_test.tex", "w") as f:
    exp1_control_vgc_test_str = llr_test_string_summary(exp1_control_vgc_test.iloc[1])
    f.write(exp1_control_vgc_test_str)
    print("exp1_control_vgc_test_str: ", exp1_control_vgc_test_str)
    
with open("./inputs/exp1_control_vgc_maineff.tex", "w") as f:
    f.write(exp1_control_vgc_maineff_sum[0])
    print("exp1_control_vgc_maineff_sum: ", exp1_control_vgc_maineff_sum[0])

exp1_control_vgc_test_str:  $\chi^2(1) = 145.39, p = 1.8 \times 10^{-33}$
exp1_control_vgc_maineff_sum:  $\beta = 0.033, \text{S.E.} = 0.003$


### Exp 2

In [33]:
%%R -i exp2_control_at -o exp2_all_control_vgc_test,exp2_all_control_vgc_maineff_sum
exp2_all_control_vgc <- lmer(
    attention_N ~ (1 | sessionId) + (1 | grid)
        + perceptual_attn + navigation_attn 
        + vgc_weight_Z,
    data=exp2_control_at,
    control=lmerControl(optimizer="bobyqa",optCtrl=list(maxfun=1e+07)),
    REML=F
)
exp2_all_control_vgc_lesioned <- update(exp2_all_control_vgc, ~ . - vgc_weight_Z)
# test if lesion is significant
exp2_all_control_vgc_test <- lrtest(
    exp2_all_control_vgc_lesioned,
    exp2_all_control_vgc
)

# get beta on vgc weight
exp2_all_control_vgc_summary <- summary(exp2_all_control_vgc)
exp2_all_control_vgc_maineff_sum <- paste(
    "$\\beta = ", round(exp2_all_control_vgc_summary$coefficients["vgc_weight_Z",]["Estimate"], digits=3),
    ", \\text{S.E.} = ", round(exp2_all_control_vgc_summary$coefficients["vgc_weight_Z",]["Std. Error"], digits=3),
    "$", sep=""
)

In [34]:
with open("./inputs/exp2_all_control_vgc_test.tex", "w") as f:
    exp2_all_control_vgc_test_str = llr_test_string_summary(exp2_all_control_vgc_test.iloc[1])
    f.write(exp2_all_control_vgc_test_str)
    print("exp2_all_control_vgc_test_str: ", exp2_all_control_vgc_test_str)
    
with open("./inputs/exp2_all_control_vgc_maineff.tex", "w") as f:
    f.write(exp2_all_control_vgc_maineff_sum[0])
    print("exp2_all_control_vgc_maineff_sum: ", exp2_all_control_vgc_maineff_sum[0])

exp2_all_control_vgc_test_str:  $\chi^2(1) = 12.20, p = 0.00048$
exp2_all_control_vgc_maineff_sum:  $\beta = 0.011, \text{S.E.} = 0.003$


### Exp 3

In [35]:
%%R -i exp3_control_mt -o exp3_acc_control_vgc_test,exp3_acc_control_vgc_maineff_sum
# accuracy
exp3_acc_control <- glmer(
    correct ~ (1 | sessionId)
        + perceptual_correct + navigation_correct
        + vgc_weight_Z,
    data=exp3_control_mt,
    control=glmerControl(optimizer="bobyqa",optCtrl=list(maxfun=1e+07)),
    family=binomial
)
exp3_acc_control_lesioned <- update(exp3_acc_control, ~ . - vgc_weight_Z)
print("Accuracy")
# test if lesion is significant
exp3_acc_control_vgc_test <- lrtest(
    exp3_acc_control_lesioned,
    exp3_acc_control
)
# get beta on vgc weight
exp3_acc_control_summary <- summary(exp3_acc_control)
exp3_acc_control_vgc_maineff_sum <- paste(
    "$\\beta = ", round(exp3_acc_control_summary$coefficients["vgc_weight_Z",]["Estimate"], digits=3),
    ", \\text{S.E.} = ", round(exp3_acc_control_summary$coefficients["vgc_weight_Z",]["Std. Error"], digits=3),
    "$", sep=""
)



[1] "Accuracy"


In [36]:
with open("./inputs/exp3_acc_control_vgc_test.tex", "w") as f:
    exp3_acc_control_vgc_test_str = llr_test_string_summary(exp3_acc_control_vgc_test.iloc[1])
    f.write(exp3_acc_control_vgc_test_str)
    print("exp3_acc_control_vgc_test_str: ", exp3_acc_control_vgc_test_str)
    
with open("./inputs/exp3_acc_control_vgc_maineff.tex", "w") as f:
    f.write(exp3_acc_control_vgc_maineff_sum[0])
    print("exp3_acc_control_vgc_maineff_sum: ", exp3_acc_control_vgc_maineff_sum[0])

exp3_acc_control_vgc_test_str:  $\chi^2(1) = 106.36, p = 6.2 \times 10^{-25}$
exp3_acc_control_vgc_maineff_sum:  $\beta = 0.58, \text{S.E.} = 0.058$


In [37]:
%%R -i exp3_control_mt -o exp3_conf_control_vgc_test,exp3_conf_control_vgc_maineff_sum
# confidence
exp3_conf_control <- lmer(
    conf_N ~ (1 | sessionId) + (1 | grid)
        + perceptual_conf + navigation_conf 
        + vgc_weight_Z,
    data=exp3_control_mt,
    control=lmerControl(optimizer="bobyqa",optCtrl=list(maxfun=1e+07)),
    REML=F
)
exp3_conf_control_lesioned <- update(exp3_conf_control, ~ . - vgc_weight_Z)
# test if lesion is significant
exp3_conf_control_vgc_test <- lrtest(
    exp3_conf_control_lesioned,
    exp3_conf_control
)
# get beta on vgc weight
exp3_conf_control_summary <- summary(exp3_conf_control)
exp3_conf_control_vgc_maineff_sum <- paste(
    "$\\beta = ", round(exp3_conf_control_summary$coefficients["vgc_weight_Z",]["Estimate"], digits=3),
    ", \\text{S.E.} = ", round(exp3_conf_control_summary$coefficients["vgc_weight_Z",]["Std. Error"], digits=3),
    "$", sep=""
)

In [38]:
with open("./inputs/exp3_conf_control_vgc_test.tex", "w") as f:
    exp3_conf_control_vgc_test_str = llr_test_string_summary(exp3_conf_control_vgc_test.iloc[1])
    f.write(exp3_conf_control_vgc_test_str)
    print("exp3_conf_control_vgc_test_str: ", exp3_conf_control_vgc_test_str)
    
with open("./inputs/exp3_conf_control_vgc_maineff.tex", "w") as f:
    f.write(exp3_conf_control_vgc_maineff_sum[0])
    print("exp3_conf_control_vgc_maineff_sum: ", exp3_conf_control_vgc_maineff_sum[0])

exp3_conf_control_vgc_test_str:  $\chi^2(1) = 18.56, p = 1.6 \times 10^{-5}$
exp3_conf_control_vgc_maineff_sum:  $\beta = 0.039, \text{S.E.} = 0.009$


In [39]:
%%R -i exp3_control_at -o exp3_attn_control_vgc_test,exp3_attn_control_vgc_maineff_sum
# awareness
exp3_attn_control <- lmer(
    attention_N ~ (1 | sessionId)
        + perceptual_attn + navigation_attn
        + vgc_weight_Z,
    data=exp3_control_at,
    control=lmerControl(optimizer="bobyqa",optCtrl=list(maxfun=1e+07)),
    REML=F
)
exp3_attn_control_lesioned <- update(exp3_attn_control, ~ . - vgc_weight_Z)
# test if lesion is significant
exp3_attn_control_vgc_test <- lrtest(
    exp3_attn_control_lesioned,
    exp3_attn_control
)
# get beta on vgc weight
exp3_attn_control_summary <- summary(exp3_attn_control)
exp3_attn_control_vgc_maineff_sum <- paste(
    "$\\beta = ", round(exp3_attn_control_summary$coefficients["vgc_weight_Z",]["Estimate"], digits=3),
    ", \\text{S.E.} = ", round(exp3_attn_control_summary$coefficients["vgc_weight_Z",]["Std. Error"], digits=3),
    "$", sep=""
)

In [40]:
with open("./inputs/exp3_attn_control_vgc_test.tex", "w") as f:
    exp3_attn_control_vgc_test_str = llr_test_string_summary(exp3_attn_control_vgc_test.iloc[1])
    f.write(exp3_attn_control_vgc_test_str)
    print("exp3_attn_control_vgc_test_str: ", exp3_attn_control_vgc_test_str)
    
with open("./inputs/exp3_attn_control_vgc_maineff.tex", "w") as f:
    f.write(exp3_attn_control_vgc_maineff_sum[0])
    print("exp3_attn_control_vgc_maineff_sum: ", exp3_attn_control_vgc_maineff_sum[0])

exp3_attn_control_vgc_test_str:  $\chi^2(1) = 55.34, p = 1.0 \times 10^{-13}$
exp3_attn_control_vgc_maineff_sum:  $\beta = 0.054, \text{S.E.} = 0.007$
